In [65]:
import numpy as np
import scipy as sp
import pandas as pd
from IPython.display import display, HTML

data = pd.read_csv("malware_Binary.csv")
display(HTML(data.head(100).to_html()))

# keep 19 columns and remove others which are not appropriate to be included in this classification task
df=data.drop(['hash','millisecond','policy','state','cached_hole_size','free_area_cache'
             ,'mm_users','end_data','last_interval','min_flt','maj_flt','fs_excl_counter'
             ,'lock','utime','stime','gtime','cgtime'],axis=1)

# print out and display dataframe as tables in HTML
display(HTML(df.head(10).to_html()))

,hash,millisecond,classification,os,state,usage_counter,prio,static_prio,normal_prio,policy,vm_pgoff,vm_truncate_count,task_size,cached_hole_size,free_area_cache,mm_users,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,end_data,last_interval,nvcsw,nivcsw,min_flt,maj_flt,fs_excl_counter,lock,utime,stime,gtime,cgtime,signal_nvcsw
0,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,0,malware,CentOS,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
1,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,1,malware,Windows,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
2,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,2,malware,Mac,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
3,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,3,malware,Ubuntu,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
4,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,4,malware,Mac,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
5,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,5,malware,Windows,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
6,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,6,malware,Ubuntu,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
7,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,7,malware,Mac,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
8,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,8,malware,CentOS,0,0,3069378560,14274,0,0,0,13173,0,0,24,724,6850,0,150,120,124,210,0,120,3473,341974,0,0,120,0,3204448256,380690,4,0,0,0
9,42fb5e2ec009a05ff5143227297074f1e9c6c3ebb9c914e223349672eca79ad0,9,malware,Mac,0,0,3069378560,14274,0,0,0,13173,0,0,25,724,6852,0,150,120,124,211,0,120,3473,341974,0,130,120,0,3204448256,380690,4,0,0,0


,classification,os,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw
0,malware,CentOS,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
1,malware,Windows,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
2,malware,Mac,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
3,malware,Ubuntu,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
4,malware,Mac,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
5,malware,Windows,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
6,malware,Ubuntu,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
7,malware,Mac,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
8,malware,CentOS,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0
9,malware,Mac,0,3069378560,14274,0,0,13173,0,6852,0,150,120,124,211,0,341974,0,0


In [74]:
import pandas as pd
print('Column Datatypes:\n',df.dtypes)

# convert all nominal variables to binary variables
df_raw=df.copy(deep=True) 
df_knn=df.copy(deep=True) 
# create new binary columns
df_dummies=pd.get_dummies(df_knn[['os']])
# add them to dataframe
df_knn=df_knn.join(df_dummies)
# drop original columns
df_knn=df_knn.drop(['os'],axis=1)
display('Data Example:',HTML(df_knn.head(10).to_html()))

# List of columns to exclude from normalization
columns_to_exclude = ['classification','usage_counter', 'normal_prio', 'vm_pgoff', 'task_size', 'hiwater_rss', 'nr_ptes', 'signal_nvcsw'] 

# Filter the columns to normalize
columns_to_normalize = [col for col in df_knn.columns if col not in columns_to_exclude]

# Min-max normalization for selected columns
for col in columns_to_normalize:
    df_knn[col] = (df_knn[col] - df_knn[col].min()) / (df_knn[col].max() - df_knn[col].min())

# Display the DataFrame
display(HTML(df_knn.head(10).to_html()))

# encode label, since KNN requires label encoding
from sklearn import preprocessing

y = df_knn['classification']
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y)

print(y_encoded)

df_knn['classification'] = y_encoded
x_features = df_knn.drop('classification',axis=1)

display(HTML(df_knn.head(10).to_html()))

Column Datatypes:
 classification       object
os                   object
usage_counter         int64
prio                  int64
static_prio           int64
normal_prio           int64
vm_pgoff              int64
vm_truncate_count     int64
task_size             int64
map_count             int64
hiwater_rss           int64
total_vm              int64
shared_vm             int64
exec_vm               int64
reserved_vm           int64
nr_ptes               int64
nvcsw                 int64
nivcsw                int64
signal_nvcsw          int64
dtype: object


'Data Example:'

,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,1,0,0,0,0
1,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,0,0,1
2,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,1,0,0
3,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,0,1,0
4,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,1,0,0
5,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,0,0,1
6,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,0,1,0
7,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,0,0,1,0,0
8,malware,0,3069378560,14274,0,0,13173,0,6850,0,150,120,124,210,0,341974,0,0,1,0,0,0,0
9,malware,0,3069378560,14274,0,0,13173,0,6852,0,150,120,124,211,0,341974,0,0,0,0,1,0,0


,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
1,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
2,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
3,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
4,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
5,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
6,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
7,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
8,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
9,malware,0,0.18254,0.016007,0,0,0.199175,0,0.166589,0,0.052031,1.0,0.307692,0.250689,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0


[1 1 1 ... 1 1 1]


,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
1,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
2,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
3,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
4,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
5,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
6,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
7,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
8,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
9,1,0,0.18254,0.016007,0,0,0.199175,0,0.166589,0,0.052031,1.0,0.307692,0.250689,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0


In [68]:
df_binary = df_knn.copy(deep=True)
numCols = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
df_numerical = df_binary.iloc[:,numCols]
df_dummy = df_binary.drop(df_binary.columns[numCols], axis=1)
display('df_numerical:',HTML(df_numerical.head(10).to_html()))
display('df_dummy:',HTML(df_dummy.head(10).to_html()))

group_names = ['L','M','H']
for col in df_numerical.columns:
    df_numerical[col] = pd.cut(df_numerical[col], 3, labels=group_names)
display('df_numerical:',HTML(df_numerical.head(10).to_html()))

df_dummies=pd.get_dummies(df_numerical)
display('df_dummies:',HTML(df_dummies.head(10).to_html()))
cols_removed = ['usage_counter_L', 'prio_L', 'static_prio_L', 'normal_prio_L', 'vm_pgoff_L', 'vm_truncate_count_L']
df_dummies = df_dummies.drop(cols_removed, axis=1)

df_binary = pd.concat([df_dummies, df_dummy], axis=1)
display('df_binary:',HTML(df_binary.head(10).to_html()))

'df_numerical:'

,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw
0,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0
1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0
2,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0
3,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0
4,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0
5,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0
6,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0
7,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0
8,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0
9,0,0.18254,0.016007,0,0,0.199175,0,0.166589,0,0.052031,1.0,0.307692,0.250689,0,0.091519,0.0,0


'df_dummy:'

,classification,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,1,1.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,1.0
2,1,0.0,0.0,1.0,0.0,0.0
3,1,0.0,0.0,0.0,1.0,0.0
4,1,0.0,0.0,1.0,0.0,0.0
5,1,0.0,0.0,0.0,0.0,1.0
6,1,0.0,0.0,0.0,1.0,0.0
7,1,0.0,0.0,1.0,0.0,0.0
8,1,1.0,0.0,0.0,0.0,0.0
9,1,0.0,0.0,1.0,0.0,0.0


C:\Users\Alan\AppData\Local\Temp\ipykernel_10812\2701346536.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numerical[col] = pd.cut(df_numerical[col], 3, labels=group_names)
C:\Users\Alan\AppData\Local\Temp\ipykernel_10812\2701346536.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numerical[col] = pd.cut(df_numerical[col], 3, labels=group_names)
C:\Users\Alan\AppData\Local\Temp\ipykernel_10812\2701346536.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

...

,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw
0,M,L,L,M,M,L,M,L,M,L,H,L,L,M,L,L,M
1,M,L,L,M,M,L,M,L,M,L,H,L,L,M,L,L,M
2,M,L,L,M,M,L,M,L,M,L,H,L,L,M,L,L,M
3,M,L,L,M,M,L,M,L,M,L,H,L,L,M,L,L,M
4,M,L,L,M,M,L,M,L,M,L,H,L,L,M,L,L,M
5,M,L,L,M,M,L,M,L,M,L,H,L,L,M,L,L,M
6,M,L,L,M,M,L,M,L,M,L,H,L,L,M,L,L,M
7,M,L,L,M,M,L,M,L,M,L,H,L,L,M,L,L,M
8,M,L,L,M,M,L,M,L,M,L,H,L,L,M,L,L,M
9,M,L,L,M,M,L,M,L,M,L,H,L,L,M,L,L,M


'df_dummies:'

,usage_counter_L,usage_counter_M,usage_counter_H,prio_L,prio_M,prio_H,static_prio_L,static_prio_M,static_prio_H,normal_prio_L,normal_prio_M,normal_prio_H,vm_pgoff_L,vm_pgoff_M,vm_pgoff_H,vm_truncate_count_L,vm_truncate_count_M,vm_truncate_count_H,task_size_L,task_size_M,task_size_H,map_count_L,map_count_M,map_count_H,hiwater_rss_L,hiwater_rss_M,hiwater_rss_H,total_vm_L,total_vm_M,total_vm_H,shared_vm_L,shared_vm_M,shared_vm_H,exec_vm_L,exec_vm_M,exec_vm_H,reserved_vm_L,reserved_vm_M,reserved_vm_H,nr_ptes_L,nr_ptes_M,nr_ptes_H,nvcsw_L,nvcsw_M,nvcsw_H,nivcsw_L,nivcsw_M,nivcsw_H,signal_nvcsw_L,signal_nvcsw_M,signal_nvcsw_H
0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0
1,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0
2,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0
3,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0
4,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0
5,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0
6,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0
7,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0
8,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0
9,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0


'df_binary:'

,usage_counter_M,usage_counter_H,prio_M,prio_H,static_prio_M,static_prio_H,normal_prio_M,normal_prio_H,vm_pgoff_M,vm_pgoff_H,vm_truncate_count_M,vm_truncate_count_H,task_size_L,task_size_M,task_size_H,map_count_L,map_count_M,map_count_H,hiwater_rss_L,hiwater_rss_M,hiwater_rss_H,total_vm_L,total_vm_M,total_vm_H,shared_vm_L,shared_vm_M,shared_vm_H,exec_vm_L,exec_vm_M,exec_vm_H,reserved_vm_L,reserved_vm_M,reserved_vm_H,nr_ptes_L,nr_ptes_M,nr_ptes_H,nvcsw_L,nvcsw_M,nvcsw_H,nivcsw_L,nivcsw_M,nivcsw_H,signal_nvcsw_L,signal_nvcsw_M,signal_nvcsw_H,classification,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,1.0,0.0,0.0,0.0,0.0
1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,0.0,0.0,1.0
2,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,1.0,0.0,0.0
3,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,0.0,1.0,0.0
4,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,1.0,0.0,0.0
5,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,0.0,0.0,1.0
6,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,0.0,1.0,0.0
7,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,1.0,0.0,0.0
8,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,1.0,0.0,0.0,0.0,0.0
9,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,1.0,0.0,0.0


In [28]:
# standarized data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_knn_std = df_knn.copy(deep=True) 
x_features = df_knn_std.loc[:, df_knn_std.columns != 'classification']
cols = x_features.columns
df_knn_std = pd.DataFrame(scaler.fit_transform(x_features), 
                          columns = cols)
df_knn_std['classification'] = y_encoded
display('df_knn_std:',HTML(df_knn_std.head(10).to_html()))

'df_knn_std:'

,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows,classification
0,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,2.002378,-0.498656,-0.499156,-0.501062,-0.501718,1
1,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,-0.499156,-0.501062,1.993151,1
2,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,2.003381,-0.501062,-0.501718,1
3,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,-0.499156,1.995760,-0.501718,1
4,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,2.003381,-0.501062,-0.501718,1
5,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,-0.499156,-0.501062,1.993151,1
6,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,-0.499156,1.995760,-0.501718,1
7,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,2.003381,-0.501062,-0.501718,1
8,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,2.002378,-0.498656,-0.499156,-0.501062,-0.501718,1
9,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507000,0.0,-0.373375,0.667258,-0.165103,0.050349,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,2.003381,-0.501062,-0.501718,1


In [75]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Display the first 10 rows of df_binary
display('df_binary:', HTML(df_binary.head(10).to_html()))


# Define your target variable 'classification'
y = df_binary['classification']

# Define your feature matrix by excluding the target variable
x = df_binary.drop('classification', axis=1)

# Initialize the Categorical Naive Bayes classifier
clf = CategoricalNB(alpha=1)

# Split the data into training (85%) and testing (15%) sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

# Define a precision scorer for cross-validation
precision = make_scorer(precision_score, average='micro')

# Perform N-fold Cross Validation on the training set and compute accuracy, precision, recall, and F1 score
acc = cross_val_score(clf, x_train, y_train, cv=5, scoring='accuracy').mean()
prec = cross_val_score(clf, x_train, y_train, cv=5, scoring=precision).mean()
recall = cross_val_score(clf, x_train, y_train, cv=5, scoring='recall_micro').mean()
f1 = cross_val_score(clf, x_train, y_train, cv=5, scoring='f1_micro').mean()

# Calculate micro-AUC during cross-validation
y_pred_probs = cross_val_predict(clf, x_train, y_train, cv=5, method='predict_proba')
micro_auc_scores = []
for i in range(y_train.nunique()):
    y_true_i = (y_train == i).astype(int)
    micro_auc = roc_auc_score(y_true_i, y_pred_probs[:, i], average='micro')
    micro_auc_scores.append(micro_auc)
    
# Calculate the average micro-AUC across all classes
average_micro_auc = np.mean(micro_auc_scores)


# Fit the classifier to the training data
clf.fit(x_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(x_test)

# Calculate evaluation metrics on the test set
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred, average='micro')
test_recall = recall_score(y_test, y_pred, average='micro')
test_f1 = f1_score(y_test, y_pred, average='micro')


print("N-fold Cross Validation:")
print("Accuracy =", test_accuracy)
print("Precision =", test_precision)
print("Recall =", test_recall)
print("F1 Score =", test_f1)
print("Micro-AUC =", average_micro_auc)

'df_binary:'

,usage_counter_M,usage_counter_H,prio_M,prio_H,static_prio_M,static_prio_H,normal_prio_M,normal_prio_H,vm_pgoff_M,vm_pgoff_H,vm_truncate_count_M,vm_truncate_count_H,task_size_L,task_size_M,task_size_H,map_count_L,map_count_M,map_count_H,hiwater_rss_L,hiwater_rss_M,hiwater_rss_H,total_vm_L,total_vm_M,total_vm_H,shared_vm_L,shared_vm_M,shared_vm_H,exec_vm_L,exec_vm_M,exec_vm_H,reserved_vm_L,reserved_vm_M,reserved_vm_H,nr_ptes_L,nr_ptes_M,nr_ptes_H,nvcsw_L,nvcsw_M,nvcsw_H,nivcsw_L,nivcsw_M,nivcsw_H,signal_nvcsw_L,signal_nvcsw_M,signal_nvcsw_H,classification,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,1.0,0.0,0.0,0.0,0.0
1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,0.0,0.0,1.0
2,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,1.0,0.0,0.0
3,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,0.0,1.0,0.0
4,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,1.0,0.0,0.0
5,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,0.0,0.0,1.0
6,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,0.0,1.0,0.0
7,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,1.0,0.0,0.0
8,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,1.0,0.0,0.0,0.0,0.0
9,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1,0.0,0.0,1.0,0.0,0.0


N-fold Cross Validation:
Accuracy = 0.7354
Precision = 0.7354
Recall = 0.7354
F1 Score = 0.7354000000000002
Micro-AUC = 0.817111967335826


In [77]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import f1_score, recall_score, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

# Initialize the BernoulliNB classifier
clf = BernoulliNB(alpha=1)

# Fit the classifier on the training data
clf.fit(X_train, y_train)

# Get predictions on the test set
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='micro')

micro_f1_scorer = make_scorer(f1_score, average='micro')
micro_recall_scorer = make_scorer(recall_score, average='micro')
micro_auc_scorer = make_scorer(roc_auc_score, average='micro', multi_class='ovr')
micro_f1 = cross_val_score(clf, x, y, cv=5, scoring=micro_f1_scorer).mean()
micro_recall = cross_val_score(clf, x, y, cv=5, scoring=micro_recall_scorer).mean()
micro_auc = cross_val_score(clf, x, y, cv=5, scoring=micro_auc_scorer).mean()

print("N-fold Cross Validation:")
print("Accuracy =", acc)
print("Precision =", prec)
print("F1 Score =", micro_f1)
print("Recall =", micro_recall)
print("Micro-AUC =", micro_auc)

N-fold Cross Validation:
Accuracy = 0.7354
Precision = 0.7354
F1 Score = 0.64874
Recall = 0.64874
Micro-AUC = 0.64874


In [85]:
from sklearn.naive_bayes import GaussianNB

display('df_num:',HTML(df_knn.head(10).to_html()))
display('df_num_std:',HTML(df_knn_std.head(10).to_html()))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

y = df_knn_std['classification']
x = df_knn_std.drop('classification', axis=1)
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='micro')

micro_f1_scorer = make_scorer(f1_score, average='micro')
micro_recall_scorer = make_scorer(recall_score, average='micro')
micro_auc_scorer = make_scorer(roc_auc_score, average='micro', multi_class='ovr')
micro_f1 = cross_val_score(clf, x_train, y_train, cv=5, scoring=micro_f1_scorer).mean()
micro_recall = cross_val_score(clf, x_train, y_train, cv=5, scoring=micro_recall_scorer).mean()
micro_auc = cross_val_score(clf, x_train, y_train, cv=5, scoring=micro_auc_scorer).mean()

print("N-fold Cross Validation:")
print("Accuracy =", acc)
print("Precision =", prec)
print("F1 Score =", micro_f1)
print("Recall =", micro_recall)
print("Micro-AUC =", micro_auc)

'df_num:'

,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
1,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
2,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
3,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
4,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
5,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
6,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
7,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
8,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
9,1,0,0.18254,0.016007,0,0,0.199175,0,0.166589,0,0.052031,1.0,0.307692,0.250689,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0


'df_num_std:'

,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows,classification
0,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,2.002378,-0.498656,-0.499156,-0.501062,-0.501718,1
1,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,-0.499156,-0.501062,1.993151,1
2,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,2.003381,-0.501062,-0.501718,1
3,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,-0.499156,1.995760,-0.501718,1
4,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,2.003381,-0.501062,-0.501718,1
5,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,-0.499156,-0.501062,1.993151,1
6,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,-0.499156,1.995760,-0.501718,1
7,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,2.003381,-0.501062,-0.501718,1
8,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507528,0.0,-0.373375,0.667258,-0.165103,0.041477,0.0,-0.695251,-0.625663,0.0,2.002378,-0.498656,-0.499156,-0.501062,-0.501718,1
9,0.0,-1.105059,-0.848177,0.0,0.0,-0.657076,0.0,-0.507000,0.0,-0.373375,0.667258,-0.165103,0.050349,0.0,-0.695251,-0.625663,0.0,-0.499406,-0.498656,2.003381,-0.501062,-0.501718,1


N-fold Cross Validation:
Accuracy = 0.7020666666666666
Precision = 0.7020666666666666
F1 Score = 0.7030705882352942
Recall = 0.7030705882352942
Micro-AUC = 0.7029895405734303


In [91]:
from sklearn.naive_bayes import MultinomialNB 

display('df_num:',HTML(df_knn.head(10).to_html()))

y = df_knn['classification']
x = df_knn.drop('classification', axis=1)
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='micro')

micro_f1_scorer = make_scorer(f1_score, average='micro')
micro_recall_scorer = make_scorer(recall_score, average='micro')
micro_auc_scorer = make_scorer(roc_auc_score, average='micro', multi_class='ovr')
micro_f1 = cross_val_score(clf, x, y, cv=5, scoring=micro_f1_scorer).mean()
micro_recall = cross_val_score(clf, x, y, cv=5, scoring=micro_recall_scorer).mean()
micro_auc = cross_val_score(clf, x, y, cv=5, scoring=micro_auc_scorer).mean()

acc=cross_val_score(clf, x, y, cv=5, scoring='accuracy').mean()
prec=cross_val_score(clf, x, y, cv=5, scoring=precision).mean()

print("N-fold Cross Validation:")
print("Accuracy =", acc)
print("Precision =", prec)
print("F1 Score =", micro_f1)
print("Recall =", micro_recall)
print("Micro-AUC =", micro_auc)

'df_num:'

,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
1,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
2,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
3,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
4,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
5,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
6,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
7,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
8,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
9,1,0,0.18254,0.016007,0,0,0.199175,0,0.166589,0,0.052031,1.0,0.307692,0.250689,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0


N-fold Cross Validation:
Accuracy = 0.6471
Precision = 0.6471
F1 Score = 0.6471
Recall = 0.6471
Micro-AUC = 0.6471


In [92]:
from sklearn.naive_bayes import ComplementNB 

display('df_num:',HTML(df_knn.head(10).to_html()))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)
y = df_knn['classification']
x = df_knn.drop('classification', axis=1)
clf = ComplementNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='micro')

micro_f1_scorer = make_scorer(f1_score, average='micro')
micro_recall_scorer = make_scorer(recall_score, average='micro')
micro_auc_scorer = make_scorer(roc_auc_score, average='micro', multi_class='ovr')
micro_f1 = cross_val_score(clf, x, y, cv=5, scoring=micro_f1_scorer).mean()
micro_recall = cross_val_score(clf, x, y, cv=5, scoring=micro_recall_scorer).mean()
micro_auc = cross_val_score(clf, x, y, cv=5, scoring=micro_auc_scorer).mean()

print("N-fold Cross Validation:")
print("Accuracy =", acc)
print("Precision =", prec)
print("F1 Score =", micro_f1)
print("Recall =", micro_recall)
print("Micro-AUC =", micro_auc)


'df_num:'

,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
1,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
2,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
3,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
4,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
5,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,0.0,1.0
6,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,0.0,1.0,0.0
7,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0
8,1,0,0.18254,0.016007,0,0,0.199175,0,0.166510,0,0.052031,1.0,0.307692,0.249311,0,0.091519,0.0,0,1.0,0.0,0.0,0.0,0.0
9,1,0,0.18254,0.016007,0,0,0.199175,0,0.166589,0,0.052031,1.0,0.307692,0.250689,0,0.091519,0.0,0,0.0,0.0,1.0,0.0,0.0


N-fold Cross Validation:
Accuracy = 0.6907333333333333
Precision = 0.6907333333333333
F1 Score = 0.6471
Recall = 0.6471
Micro-AUC = 0.6471
